# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [8]:
# Importing libraries
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.probability import FreqDist

In [3]:
# your code here
ds = pd.read_csv('Sentiment140.csv')
ds

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [4]:
# Getting a sample of the dataset as advised
ds_sample = ds.sample(20000)
ds_sample

,target,id,date,flag,user,text
1545969,4,2182298309,Mon Jun 15 12:59:16 PDT 2009,NO_QUERY,digiqr,"@JasnaPaka modp_b64 + něco ve webkit,, Firefox..."
49401,0,1678132167,Sat May 02 05:37:43 PDT 2009,NO_QUERY,msmercie,@cutthroat31 down wit the bulls! LOL
837006,4,1558646261,Sun Apr 19 09:00:01 PDT 2009,NO_QUERY,pato_30stm,@30STMWithJared yes! here i am how are you to...
1037252,4,1956483534,Thu May 28 21:58:48 PDT 2009,NO_QUERY,jesseluna,@angels510 You're welcome. I've always glad t...
527915,0,2194835804,Tue Jun 16 10:19:32 PDT 2009,NO_QUERY,kellbot,@stellaloella sad txt me this afternoon if yo...
...,...,...,...,...,...,...
455137,0,2070835514,Sun Jun 07 18:22:49 PDT 2009,NO_QUERY,FahimFarook,"OK, maybe not on the Pre .... apparently devel..."
918165,4,1753615446,Sun May 10 02:32:04 PDT 2009,NO_QUERY,roweeeena,@sarahfloss went with john grisham instead... ...
699493,0,2254466730,Sat Jun 20 09:47:34 PDT 2009,NO_QUERY,akarshsimha,@louiswu: Wow. Have a nice time. Both me and A...
1502789,4,2071773702,Sun Jun 07 19:54:26 PDT 2009,NO_QUERY,Niko_xo,Could Twitter all day if it wasn't most likely...


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [5]:
# Functions from previous challenge
def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    return re.sub(r'  *', ' ', re.sub(r'[^a-z]', ' ', 
                                      re.sub(r'www\.\S*', ' ', re.sub(r'http[s]?://\S*', ' ', s.lower())))).strip()


def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    return nltk.word_tokenize(s)


def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    ps = PorterStemmer()
    wnl = WordNetLemmatizer()
    
    stemmed_list = [ps.stem(w) for w in l]
    lemmed_on_stemmed_list = [wnl.lemmatize(w) for w in stemmed_list]
    
    return lemmed_on_stemmed_list


def remove_stopwords(lst, lang = 'english'):
    """
    Remove English (default) stopwords from a list of strings.

    Args:
        lst: A list of strings.
        l = Language of the stopwords.

    Returns:
        A list of strings after stop words are removed.
    """
    stop_words = stopwords.words(lang)
    return [word for word in lst if word not in stop_words]

In [220]:
# your code here
#ds_sample['text_processed'] = ds_sample['text'].apply(clean_up)
#ds_sample['text_processed'] = ds_sample['text_processed'].apply(tokenize)
#ds_sample['text_processed'] = ds_sample['text_processed'].apply(stem_and_lemmatize)
#ds_sample['text_processed'] = ds_sample['text_processed'].apply(remove_stopwords)

# Looks like it worked. Can I do all 4 of these in the same line? Saw some people doing it but I didn't get it.
## some did in reverse order like 1st remove_stopwords and going back?
### might've answered myself when I rewrote the clean_up func before

In [6]:
# Ok, I decided to try and it worked :D
## I'll still take the explanation though :3
ds_sample['text_processed'] = \
ds_sample['text'].apply(lambda x: remove_stopwords(stem_and_lemmatize(tokenize(clean_up(x)))))
ds_sample

,target,id,date,flag,user,text,text_processed
1545969,4,2182298309,Mon Jun 15 12:59:16 PDT 2009,NO_QUERY,digiqr,"@JasnaPaka modp_b64 + něco ve webkit,, Firefox...","[jasnapaka, modp, b, n, co, webkit, firefox, v..."
49401,0,1678132167,Sat May 02 05:37:43 PDT 2009,NO_QUERY,msmercie,@cutthroat31 down wit the bulls! LOL,"[cutthroat, wit, bull, lol]"
837006,4,1558646261,Sun Apr 19 09:00:01 PDT 2009,NO_QUERY,pato_30stm,@30STMWithJared yes! here i am how are you to...,"[stmwithjar, ye, today]"
1037252,4,1956483534,Thu May 28 21:58:48 PDT 2009,NO_QUERY,jesseluna,@angels510 You're welcome. I've always glad t...,"[angel, welcom, alway, glad, suit, someon, hel..."
527915,0,2194835804,Tue Jun 16 10:19:32 PDT 2009,NO_QUERY,kellbot,@stellaloella sad txt me this afternoon if yo...,"[stellaloella, sad, txt, thi, afternoon, got, ..."
...,...,...,...,...,...,...,...
455137,0,2070835514,Sun Jun 07 18:22:49 PDT 2009,NO_QUERY,FahimFarook,"OK, maybe not on the Pre .... apparently devel...","[ok, mayb, pre, appar, develop, onli, get, acc..."
918165,4,1753615446,Sun May 10 02:32:04 PDT 2009,NO_QUERY,roweeeena,@sarahfloss went with john grisham instead... ...,"[sarahfloss, went, john, grisham, instead, wa,..."
699493,0,2254466730,Sat Jun 20 09:47:34 PDT 2009,NO_QUERY,akarshsimha,@louiswu: Wow. Have a nice time. Both me and A...,"[louiswu, wow, nice, time, arun, thi, time, gc..."
1502789,4,2071773702,Sun Jun 07 19:54:26 PDT 2009,NO_QUERY,Niko_xo,Could Twitter all day if it wasn't most likely...,"[could, twitter, day, like, annoy, folow]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [9]:
bow = [word for lst in ds_sample['text_processed'] for word in lst]
fdist = FreqDist(bow)
most_common = fdist.most_common(5000)  # [w[0] for w in fdist.most_common(5000)] in case I just want the words
most_common

[('go', 1721),
 ('get', 1402),
 ('day', 1328),
 ('wa', 1302),
 ('thi', 1174),
 ('work', 1112),
 ('good', 1105),
 ('love', 1042),
 ('like', 1013),
 ('u', 957),
 ('quot', 900),
 ('got', 896),
 ('today', 852),
 ('time', 820),
 ('lol', 759),
 ('want', 755),
 ('thank', 726),
 ('miss', 720),
 ('know', 689),
 ('feel', 680),
 ('one', 680),
 ('back', 672),
 ('think', 647),
 ('im', 646),
 ('realli', 640),
 ('see', 637),
 ('hope', 600),
 ('amp', 597),
 ('night', 564),
 ('watch', 553),
 ('still', 550),
 ('new', 547),
 ('well', 542),
 ('need', 523),
 ('make', 517),
 ('na', 517),
 ('home', 511),
 ('ha', 488),
 ('oh', 473),
 ('come', 462),
 ('much', 462),
 ('last', 460),
 ('look', 454),
 ('sleep', 450),
 ('morn', 449),
 ('twitter', 442),
 ('tomorrow', 441),
 ('wait', 422),
 ('wish', 416),
 ('great', 410),
 ('right', 409),
 ('haha', 405),
 ('fun', 381),
 ('sad', 376),
 ('happi', 367),
 ('follow', 365),
 ('onli', 362),
 ('tri', 358),
 ('week', 357),
 ('veri', 355),
 ('whi', 345),
 ('would', 342),
 ('ba

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [223]:
# your code here
# Mattia's
def find_features(document):
    words = set(document)
    features = {}
    for w in most_common: # word_features
        features[w] = w in words  # tried a few changes here, but output seems the same
    return features

                    #ds_sample['text_processed'] was originally here instead of just the 'text' one
documents = list(zip(ds_sample['text'], np.where(ds_sample['target'] == 4, True, False)))

# The output isn't exactly what we want, I think
featuresets = [(find_features(rev)) for (rev, category) in documents]  # tried few changes but output still not good

In [224]:
featuresets[0]

{('go', 1761): False,
 ('get', 1389): False,
 ('day', 1386): False,
 ('wa', 1296): False,
 ('good', 1131): False,
 ('thi', 1123): False,
 ('work', 1099): False,
 ('love', 1075): False,
 ('like', 1051): False,
 ('u', 915): False,
 ('quot', 913): False,
 ('got', 859): False,
 ('time', 832): False,
 ('today', 775): False,
 ('one', 768): False,
 ('thank', 755): False,
 ('lol', 746): False,
 ('want', 743): False,
 ('back', 675): False,
 ('see', 672): False,
 ('know', 660): False,
 ('miss', 645): False,
 ('feel', 627): False,
 ('realli', 621): False,
 ('think', 621): False,
 ('amp', 613): False,
 ('im', 576): False,
 ('well', 565): False,
 ('still', 555): False,
 ('hope', 554): False,
 ('make', 550): False,
 ('watch', 546): False,
 ('new', 544): False,
 ('need', 537): False,
 ('night', 536): False,
 ('home', 516): False,
 ('na', 507): False,
 ('ha', 497): False,
 ('come', 496): False,
 ('look', 492): False,
 ('oh', 476): False,
 ('much', 464): False,
 ('twitter', 439): False,
 ('wish', 438):

In [10]:
# your code here
# Jo's version seems to be the correct implementation of what's asked in this exercise, I believe
def find_features(document, bow):
    text = document.lower()
    features = dict()
    for w, c in bow:
        features[w] = w in text
    return features

def make_matrix(series_text, series_target, bow):
    return [(find_features(s, bow), bool(t)) for s, t in zip(series_text.values, series_target.values)]

matrix = make_matrix(ds_sample['text'], ds_sample['target'], most_common)

In [11]:
matrix[0] # leaving it like this [0] otherwise it makes it impossible to do anything as it becomes too heavy

({'go': False,
  'get': False,
  'day': False,
  'wa': False,
  'thi': False,
  'work': False,
  'good': False,
  'love': False,
  'like': False,
  'u': True,
  'quot': False,
  'got': False,
  'today': False,
  'time': False,
  'lol': False,
  'want': False,
  'thank': False,
  'miss': False,
  'know': False,
  'feel': False,
  'one': False,
  'back': False,
  'think': False,
  'im': False,
  'realli': False,
  'see': False,
  'hope': False,
  'amp': False,
  'night': False,
  'watch': False,
  'still': False,
  'new': False,
  'well': False,
  'need': False,
  'make': False,
  'na': True,
  'home': False,
  'ha': False,
  'oh': False,
  'come': False,
  'much': False,
  'last': False,
  'look': False,
  'sleep': False,
  'morn': False,
  'twitter': False,
  'tomorrow': False,
  'wait': False,
  'wish': False,
  'great': False,
  'right': False,
  'haha': False,
  'fun': False,
  'sad': False,
  'happi': False,
  'follow': False,
  'onli': False,
  'tri': False,
  'week': False,
  've

### Building and Training Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [12]:
# your code here
# Splitting into training and testing set
len(matrix) * 0.2  # 4000
training_set = matrix[:4000]  # 20% of the data
testing_set = matrix[4000:]  # 80% of the data

classifier = nltk.NaiveBayesClassifier.train(training_set)
classifier.show_most_informative_features(10)

Most Informative Features
                    hurt = True            False : True   =     24.3 : 1.0
                    poor = True            False : True   =     15.0 : 1.0
                     sad = True            False : True   =     13.8 : 1.0
                    sick = True            False : True   =     10.7 : 1.0
                  missin = True            False : True   =      9.8 : 1.0
                 appoint = True            False : True   =      9.7 : 1.0
                   smile = True             True : False  =      9.0 : 1.0
                     btw = True             True : False  =      7.7 : 1.0
              disappoint = True            False : True   =      7.7 : 1.0
                    dead = True            False : True   =      7.7 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [13]:
# your code here
# This takes forever to run!!
print('Model accuracy percent:', nltk.classify.accuracy(classifier, testing_set) * 100)  # so bad :3

Model accuracy percent: 68.1875


In [31]:
print('coco:', str(round(0.2567 * 100, 2)) + '%')

coco: 25.67%


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [32]:
# your code here
# Increasing the sample size by 30k. Should help increase the accuracy score, I hope
ds_sample2 = ds.sample(50000)
ds_sample2

,target,id,date,flag,user,text
1593907,4,2192040161,Tue Jun 16 06:24:30 PDT 2009,NO_QUERY,sncanter,"dentist, hangin out, first real choreography j..."
39541,0,1573554528,Tue Apr 21 01:43:35 PDT 2009,NO_QUERY,TexasGirlSari,Well off to bed I go for real this time LOL! N...
1449574,4,2062853926,Sun Jun 07 00:31:20 PDT 2009,NO_QUERY,GeekyGreeky87,@Rican413 im glad to hear you're ok.
1277434,4,2001321357,Tue Jun 02 00:30:34 PDT 2009,NO_QUERY,princesstimetoy,@kitrona Could be - these are Adult capes - RE...
1479646,4,2066715432,Sun Jun 07 11:02:24 PDT 2009,NO_QUERY,SongoftheOss,Thanks to @djilo @D_Doyle @Stanleydavis @sdros...
...,...,...,...,...,...,...
837723,4,1558854921,Sun Apr 19 09:35:17 PDT 2009,NO_QUERY,wanderblah,@axon ah! you're in office too! nice
792691,0,2326367559,Thu Jun 25 06:56:36 PDT 2009,NO_QUERY,kawaiicrochet,Crap: less than week till my exam and I haven'...
308070,0,2000613120,Mon Jun 01 22:31:13 PDT 2009,NO_QUERY,SteviNicole,is having NO luck finding a roommate.
1450799,4,2062974674,Sun Jun 07 00:56:04 PDT 2009,NO_QUERY,MiszJaxon,@tappdancekidd well im calling it a night or m...


In [33]:
# Preparing the text for the analysis
ds_sample2['text_processed'] = \
ds_sample2['text'].apply(lambda x: remove_stopwords(stem_and_lemmatize(tokenize(clean_up(x)))))
ds_sample2

,target,id,date,flag,user,text,text_processed
1593907,4,2192040161,Tue Jun 16 06:24:30 PDT 2009,NO_QUERY,sncanter,"dentist, hangin out, first real choreography j...","[dentist, hangin, first, real, choreographi, j..."
39541,0,1573554528,Tue Apr 21 01:43:35 PDT 2009,NO_QUERY,TexasGirlSari,Well off to bed I go for real this time LOL! N...,"[well, bed, go, real, thi, time, lol, noth, do..."
1449574,4,2062853926,Sun Jun 07 00:31:20 PDT 2009,NO_QUERY,GeekyGreeky87,@Rican413 im glad to hear you're ok.,"[rican, im, glad, hear, ok]"
1277434,4,2001321357,Tue Jun 02 00:30:34 PDT 2009,NO_QUERY,princesstimetoy,@kitrona Could be - these are Adult capes - RE...,"[kitrona, could, adult, cape, realli, long, me..."
1479646,4,2066715432,Sun Jun 07 11:02:24 PDT 2009,NO_QUERY,SongoftheOss,Thanks to @djilo @D_Doyle @Stanleydavis @sdros...,"[thank, djilo, doyl, stanleydavi, sdrosenb, pr..."
...,...,...,...,...,...,...,...
837723,4,1558854921,Sun Apr 19 09:35:17 PDT 2009,NO_QUERY,wanderblah,@axon ah! you're in office too! nice,"[axon, ah, offic, nice]"
792691,0,2326367559,Thu Jun 25 06:56:36 PDT 2009,NO_QUERY,kawaiicrochet,Crap: less than week till my exam and I haven'...,"[crap, le, week, till, exam, start, lesson, ye..."
308070,0,2000613120,Mon Jun 01 22:31:13 PDT 2009,NO_QUERY,SteviNicole,is having NO luck finding a roommate.,"[luck, find, roommat]"
1450799,4,2062974674,Sun Jun 07 00:56:04 PDT 2009,NO_QUERY,MiszJaxon,@tappdancekidd well im calling it a night or m...,"[tappdancekidd, well, im, call, night, morn, l..."


In [34]:
# Creating the bag of words
bow = [word for lst in ds_sample2['text_processed'] for word in lst]
fdist = FreqDist(bow)

# Reducing the top feature number to train the model faster. Don't know if this increases the score or not
## on the 2nd run decided to try with 5k again and not 3k
most_common = fdist.most_common(5000)
most_common

[('go', 4421),
 ('day', 3430),
 ('get', 3427),
 ('wa', 3261),
 ('thi', 3060),
 ('good', 2861),
 ('work', 2795),
 ('like', 2586),
 ('love', 2543),
 ('quot', 2375),
 ('got', 2242),
 ('u', 2203),
 ('today', 2183),
 ('time', 2130),
 ('lol', 1877),
 ('want', 1850),
 ('one', 1839),
 ('back', 1835),
 ('miss', 1817),
 ('thank', 1795),
 ('know', 1700),
 ('see', 1696),
 ('feel', 1593),
 ('think', 1569),
 ('im', 1517),
 ('realli', 1516),
 ('amp', 1497),
 ('still', 1386),
 ('watch', 1386),
 ('night', 1358),
 ('new', 1353),
 ('make', 1350),
 ('hope', 1348),
 ('well', 1334),
 ('need', 1309),
 ('na', 1302),
 ('ha', 1273),
 ('home', 1271),
 ('oh', 1236),
 ('look', 1212),
 ('come', 1156),
 ('morn', 1130),
 ('much', 1107),
 ('last', 1090),
 ('twitter', 1085),
 ('tomorrow', 1070),
 ('wait', 1039),
 ('wish', 1025),
 ('sleep', 1024),
 ('haha', 1021),
 ('great', 1011),
 ('week', 929),
 ('fun', 925),
 ('sad', 903),
 ('onli', 892),
 ('would', 890),
 ('whi', 885),
 ('bad', 870),
 ('right', 865),
 ('follow', 85

In [35]:
# Building the features
def find_features(document, bow):
    text = document.lower()
    features = dict()
    for w, c in bow:
        features[w] = w in text
    return features

def make_matrix(series_text, series_target, bow):
    return [(find_features(s, bow), bool(t)) for s, t in zip(series_text.values, series_target.values)]

matrix = make_matrix(ds_sample2['text'], ds_sample2['target'], most_common)

In [42]:
# Testing the model
size = int(len(matrix) * 0.2)  # 10000 -> 20% of the data

# Increased the size of the training set to 25% of the data
#training_set = matrix[:12500]
#testing_set = matrix[12500:]

# Tried inversing the order and training with 80% of the data and testing against the 20% left
training_set = matrix[size:]
testing_set = matrix[:size]

classifier = nltk.NaiveBayesClassifier.train(training_set)

# Showing the top 15 most informative features this time
classifier.show_most_informative_features(15)

# The 'no' with 6 'o' is more negative than the one with 7 'o' :D

Most Informative Features
                 noooooo = True            False : True   =     22.3 : 1.0
                nooooooo = True            False : True   =     17.6 : 1.0
                 allergi = True            False : True   =     17.6 : 1.0
                unfortun = True            False : True   =     17.4 : 1.0
                  cancel = True            False : True   =     14.8 : 1.0
                 depress = True            False : True   =     13.6 : 1.0
            followfriday = True             True : False  =     13.4 : 1.0
                  allerg = True            False : True   =     13.0 : 1.0
                 firefox = True            False : True   =     13.0 : 1.0
                 congrat = True             True : False  =     12.6 : 1.0
                     smh = True            False : True   =     12.3 : 1.0
                    poor = True            False : True   =     11.9 : 1.0
               disappear = True            False : True   =     11.6 : 1.0

In [43]:
# Original result was 0.7137333333333333 with 50k sample, 3k top features and 20% data trained vs tested against 80%
print('Model accuracy:', str(round(nltk.classify.accuracy(classifier, testing_set) * 100, 2)) + '%')

# Wow, even though I trained it with 80% of the data, it still kept pretty much the same score as before...

Model accuracy: 71.87%


## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here